In [1]:
%env PYTHONWARNINGS=ignore
import warnings
warnings.filterwarnings("ignore")

env: PYTHONWARNINGS=ignore


In [3]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
from nemo_curator import DomainClassifier
from nemo_curator.datasets import DocumentDataset
#from nemo_curator.utils.distributed_utils import start_dask_gpu_local_cluster

In [4]:
cluster = LocalCUDACluster(rmm_async=True, rmm_pool_size="1GB")
client = Client(cluster)

In [5]:
labels = [
    "Adult",
    "Arts_and_Entertainment",
    "Autos_and_Vehicles",
    "Beauty_and_Fitness",
    "Books_and_Literature",
    "Business_and_Industrial",
    "Computers_and_Electronics",
    "Finance",
    "Food_and_Drink",
    "Games",
    "Health",
    "Hobbies_and_Leisure",
    "Home_and_Garden",
    "Internet_and_Telecom",
    "Jobs_and_Education",
    "Law_and_Government",
    "News",
    "Online_Communities",
    "People_and_Society",
    "Pets_and_Animals",
    "Real_Estate",
    "Science",
    "Sensitive_Subjects",
    "Shopping",
    "Sports",
    "Travel_and_Transportation",
]

model_file_name = "/home/nfs/syurick/LLM_domain_classifier_inference/GoogleDebertaAgree_v3b_bce_maxlen512_bs64_noRef_best.pth"
# Input can be a string or list
input_file_path = "/home/nfs/syurick/LLM_domain_classifier_inference/4360_results_jsonl_dir/"
output_file_path = "/raid/vjawa/output_file.json"


In [6]:
%%time

input_dataset = DocumentDataset.read_json(
    input_file_path, backend="cudf", add_filename=True
)

domain_classifier = DomainClassifier(
    model_file_name=model_file_name,
    labels=labels,
    batch_size=256,
)
result_dataset = domain_classifier(dataset=input_dataset)
result_dataset.to_json(output_file_dir=output_file_path, write_to_filename=True)

Reading 16 files
Starting domain classifier inference


GPU: 0, Part: 11: 100%|██████████| 937/937 [00:15<00:00, 60.47it/s]


Writing to disk complete for 16 partitions
CPU times: user 11.4 s, sys: 11.4 s, total: 22.7 s
Wall time: 49.7 s


In [7]:
output_dataset = DocumentDataset.read_json(output_file_path, backend="cudf", add_filename=True)

Reading 16 files


In [8]:
output_dataset.df

,adlr_id,filename,id,labels,pred,source_id,split_id,text,url
npartitions=16,,,,,,,,,
,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [9]:
output_dataset.df.head(10)

,adlr_id,filename,id,labels,pred,source_id,split_id,text,url
0,cc-2022-40-0431053204,00.jsonl,a8083fe4-525d-4888-8513-b91f43bd8ee1,Online_Communities,Online_Communities,crawl-data-CC-MAIN-2022-40-segments-1664030336...,lambada-0003225258-0000,Having been a community leader—and member—for ...,https://lisalarter.com/7-tips-for-building-ste...
1,cc-2022-40-0510168267,00.jsonl,559febdc-cb7f-4217-897a-c8dac325123b,Finance,Finance,crawl-data-CC-MAIN-2022-40-segments-1664030337...,lambada-0003918122-0000,Zelle is a way of sending money to almost anyo...,https://oregonmassageandwellnessclinic.com/app...
2,cc-2022-40-0695312978,00.jsonl,b1ec1a9f-693e-4672-b485-54f48a3dfdb6,Arts_and_Entertainment,Arts_and_Entertainment,crawl-data-CC-MAIN-2022-40-segments-1664030337...,lambada-0005286343-0000,Nicole Scherzinger and Enrique Lglesias Get St...,https://menzmag.com/entertainment/celebrity-go...
3,cc-2022-40-0318121708,00.jsonl,f1217f04-58d3-4c88-8d33-250401b219f6,Internet_and_Telecom,Internet_and_Telecom,crawl-data-CC-MAIN-2022-40-segments-1664030335...,lambada-0002386272-0000,Thanksgiving 2021 WhatsApp Status Video to Dow...,https://nonstop-news.com/lifestyle/thanksgivin...
4,cc-2022-40-0602859436,00.jsonl,d255ebe4-0601-469b-a5d3-c4102d83dabd,Games,Games,crawl-data-CC-MAIN-2022-40-segments-1664030337...,lambada-0004541139-0000,Lakeside Inn And Casino Lake Tahoe – Online si...,https://psplondon.com/lakeside-inn-and-casino-...
5,cc-2022-40-0025406361,00.jsonl,5d598bfa-ca17-4203-800c-4d02072c3b87,Books_and_Literature,Books_and_Literature,crawl-data-CC-MAIN-2022-40-segments-1664030334...,lambada-0000190248-0000,A THOUSAND WORDS - Alex Waterhouse-Hayward's b...,http://blog.alexwaterhousehayward.com/2006/03/...
6,cc-2022-40-0605292636,00.jsonl,42ced198-6cdb-4ef2-bf4d-dc1254da0da6,Shopping,Beauty_and_Fitness,crawl-data-CC-MAIN-2022-40-segments-1664030337...,lambada-0004601177-0000,Search our store\n\nCLOCKWORK ORANGE OUTFIT\n\...,https://dressx.com/products/clockwork-orange-o...
7,cc-2022-40-0270701137,00.jsonl,a0bbffa6-670d-43e4-8027-9fc0862df95f,News,News,crawl-data-CC-MAIN-2022-40-segments-1664030335...,lambada-0002122651-0000,The Democrat Police State Imposes its Tyranny\...,https://www.paulcraigroberts.org/2022/08/13/th...
8,cc-2022-40-0130518751,00.jsonl,80948f1a-0970-4bc4-879a-22725a388d62,Games,Games,crawl-data-CC-MAIN-2022-40-segments-1664030334...,lambada-0000961821-0000,How to Play the Lottery Online\n\nThe lottery ...,https://moellerdog.com/index.php/2022/09/16/ho...
9,cc-2022-40-0430464926,00.jsonl,234d085c-f735-4b2e-bcfd-edc65fb4ed22,Beauty_and_Fitness,Beauty_and_Fitness,crawl-data-CC-MAIN-2022-40-segments-1664030336...,lambada-0003227706-0000,LASER LIPOSUCTION\n\nLaser Liposuction works b...,https://shapechicagoland.com/services/body-tre...
